In [2]:
%matplotlib inline

In [10]:
## Uncomment If running this from remote **Still working out Qt X server stuff
#import pyvista as pv
#pv.start_xvfb()

#!export MNE_3D_OPTION_ANTIALIAS=false
#!export MESA_GL_VERSION_OVERRIDE=3.3

#%matplotlib widget

#!glxinfo | grep "OpenGL core profile version"
#!mne sys_info

OpenGL core profile version string: 3.3 (Core Profile) Mesa 20.1.4
Platform:      Linux-4.18.0-240.22.1.el8_3.x86_64-x86_64-with-centos-8.3.2011
Python:        3.7.10 (default, Feb 26 2021, 18:47:35)  [GCC 7.3.0]
Executable:    /home/rnwick/miniconda3/envs/mne-python/bin/python
CPU:           x86_64: 12 cores
Memory:        7.6 GB

mne:           0.22.0
numpy:         1.20.2 {blas=openblas, lapack=openblas}
scipy:         1.6.2
matplotlib:    3.3.4 {backend=module://ipykernel.pylab.backend_inline}

sklearn:       0.24.2
numba:         0.53.1
nibabel:       3.2.1
nilearn:       0.7.1
dipy:          1.4.1
cupy:          Not found
pandas:        1.2.4
mayavi:        Not found
pyvista:       0.30.1 {pyvistaqt=0.4.0, OpenGL 3.3 (Core Profile) Mesa 20.1.4 via llvmpipe (LLVM 10.0.0, 128 bits)}
vtk:           9.0.1
PyQt5:         5.12.3



# Corrupt known signal with point spread

The aim of this tutorial is to demonstrate how to put a known signal at a
desired location(s) in a :class:`mne.SourceEstimate` and then corrupt the
signal with point-spread by applying a forward and inverse solution.


In [3]:
import os.path as op

import numpy as np

import mne
from mne.datasets import sample

from mne.minimum_norm import read_inverse_operator, apply_inverse
from mne.simulation import simulate_stc, simulate_evoked

First, we set some parameters.



In [4]:
seed = 42

# parameters for inverse method
method = 'sLORETA'
snr = 3.
lambda2 = 1.0 / snr ** 2

# signal simulation parameters
# do not add extra noise to the known signals
nave = np.inf
T = 100
times = np.linspace(0, 1, T)
dt = times[1] - times[0]

# Paths to MEG data
data_path = ('../../../../data/tutorials/MNE-sample-data')
subjects_dir = op.join(data_path, 'subjects')
fname_fwd = op.join(data_path, 'MEG', 'sample',
                    'sample_audvis-meg-oct-6-fwd.fif')
fname_inv = op.join(data_path, 'MEG', 'sample',
                    'sample_audvis-meg-oct-6-meg-fixed-inv.fif')

fname_evoked = op.join(data_path, 'MEG', 'sample',
                       'sample_audvis-ave.fif')

print(subjects_dir)

../../../../data/tutorials/MNE-sample-data\subjects


## Load the MEG data



In [5]:
fwd = mne.read_forward_solution(fname_fwd)
fwd = mne.convert_forward_solution(fwd, force_fixed=True, surf_ori=True,
                                   use_cps=False)
fwd['info']['bads'] = []
inv_op = read_inverse_operator(fname_inv)

raw = mne.io.read_raw_fif(op.join(data_path, 'MEG', 'sample',
                                  'sample_audvis_raw.fif'))
raw.set_eeg_reference(projection=True)
events = mne.find_events(raw)
event_id = {'Auditory/Left': 1, 'Auditory/Right': 2}
epochs = mne.Epochs(raw, events, event_id, baseline=(None, 0), preload=True)
epochs.info['bads'] = []
evoked = epochs.average()

labels = mne.read_labels_from_annot('sample', subjects_dir=subjects_dir)
label_names = [label.name for label in labels]
n_labels = len(labels)

Reading forward solution from ../../../../data/tutorials/MNE-sample-data\MEG\sample\sample_audvis-meg-oct-6-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (7498 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
Reading inverse operator decomposition from ../../../../data/tutorials/MNE-sample-data\MEG\sample\sample_audvis-meg-oct-6-meg-fixed-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    305 x 305 full cov

## Estimate the background noise covariance from the baseline period



In [6]:
cov = mne.compute_covariance(epochs, tmin=None, tmax=0.)

Computing rank from data with rank=None
    Using tolerance 1.4e-08 (2.2e-16 eps * 306 dim * 2.1e+05  max singular value)
    Estimated rank (mag + grad): 303
    MEG: rank 303 computed from 306 data channels with 3 projectors
    Using tolerance 4.9e-11 (2.2e-16 eps * 60 dim * 3.7e+03  max singular value)
    Estimated rank (eeg): 59
    EEG: rank 59 computed from 60 data channels with 1 projector
    Created an SSP operator (subspace dimension = 4)
    Setting small MEG eigenvalues to zero (without PCA)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 366 -> 362
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 17545
[done]


## Generate sinusoids in two spatially distant labels



In [7]:
# The known signal is all zero-s off of the two labels of interest
signal = np.zeros((n_labels, T))
idx = label_names.index('inferiorparietal-lh')
signal[idx, :] = 1e-7 * np.sin(5 * 2 * np.pi * times)
idx = label_names.index('rostralmiddlefrontal-rh')
signal[idx, :] = 1e-7 * np.sin(7 * 2 * np.pi * times)

## Find the center vertices in source space of each label

We want the known signal in each label to only be active at the center. We
create a mask for each label that is 1 at the center vertex and 0 at all
other vertices in the label. This mask is then used when simulating
source-space data.



In [8]:
hemi_to_ind = {'lh': 0, 'rh': 1}
for i, label in enumerate(labels):
    # The `center_of_mass` function needs labels to have values.
    labels[i].values.fill(1.)

    # Restrict the eligible vertices to be those on the surface under
    # consideration and within the label.
    surf_vertices = fwd['src'][hemi_to_ind[label.hemi]]['vertno']
    restrict_verts = np.intersect1d(surf_vertices, label.vertices)
    com = labels[i].center_of_mass(subject='sample',
                                   subjects_dir=subjects_dir,
                                   restrict_vertices=restrict_verts,
                                   surf='white')

    # Convert the center of vertex index from surface vertex list to Label's
    # vertex list.
    cent_idx = np.where(label.vertices == com)[0][0]

    # Create a mask with 1 at center vertex and zeros elsewhere.
    labels[i].values.fill(0.)
    labels[i].values[cent_idx] = 1.

## Create source-space data with known signals

Put known signals onto surface vertices using the array of signals and
the label masks (stored in labels[i].values).



In [9]:
stc_gen = simulate_stc(fwd['src'], labels, signal, times[0], dt,
                       value_fun=lambda x: x)

## Plot original signals

Note that the original signals are highly concentrated (point) sources.




In [10]:
kwargs = dict(subjects_dir=subjects_dir, hemi='split', smoothing_steps=4,
              time_unit='s', initial_time=0.05, size=1200,
              views=['lat', 'med'])
clim = dict(kind='value', pos_lims=[1e-9, 1e-8, 1e-7])
#brain_gen = stc_gen.plot(clim=clim, **kwargs)

## Simulate sensor-space signals

Use the forward solution and add Gaussian noise to simulate sensor-space
(evoked) data from the known source-space signals. The amount of noise is
controlled by ``nave`` (higher values imply less noise).




In [13]:
method = 'sLORETA'

evoked_gen = simulate_evoked(fwd, stc_gen, evoked.info, cov, nave,
                             random_state=seed)

# Map the simulated sensor-space data to source-space using the inverse
# operator.
stc_inv = apply_inverse(evoked_gen, inv_op, lambda2, method=method)

Projecting source estimate to sensor space...
[done]
4 projection items deactivated
Created an SSP operator (subspace dimension = 3)
4 projection items activated
SSP projectors applied...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Applying inverse operator to ""...
    Picked 305 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  99.7% variance
    sLORETA...
[done]


## Plot the point-spread of corrupted signal

Notice that after applying the forward- and inverse-operators to the known
point sources that the point sources have spread across the source-space.
This spread is due to the minimum norm solution so that the signal leaks to
nearby vertices with similar orientations so that signal ends up crossing the
sulci and gyri.



In [14]:
brain_inv = stc_inv.plot(**kwargs)

## Saving videos broken at the moment
#brain_inv.save_movie('test.ffmpeg')



Using control points [0.45968308 0.57021267 1.69354621]


## Exercises
   - Change the ``method`` parameter to either ``'dSPM'`` or ``'MNE'`` to
     explore the effect of the inverse method.
   - Try setting ``evoked_snr`` to a small, finite value, e.g. 3., to see the
     effect of noise.



# Method as dSPM

In [19]:
seed = 42

# parameters for inverse method
method = 'dSPM'
snr = 3.
lambda2 = 1.0 / snr ** 2

evoked_gen = simulate_evoked(fwd, stc_gen, evoked.info, cov, nave,
                             random_state=seed)

# Map the simulated sensor-space data to source-space using the inverse
# operator.
stc_inv = apply_inverse(evoked_gen, inv_op, lambda2, method=method)
brain_inv = stc_inv.plot(**kwargs)

Projecting source estimate to sensor space...
[done]
4 projection items deactivated
Created an SSP operator (subspace dimension = 3)
4 projection items activated
SSP projectors applied...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Applying inverse operator to ""...
    Picked 305 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  96.0% variance
    sLORETA...
[done]
Using control points [1.3776512 1.6437969 4.3271975]


# Method as MNE

In [12]:
seed = 42

# parameters for inverse method
method = 'MNE'
snr = 3.
lambda2 = 1.0 / snr ** 2

evoked_gen = simulate_evoked(fwd, stc_gen, evoked.info, cov, nave,
                             random_state=seed)

# Map the simulated sensor-space data to source-space using the inverse
# operator.
stc_inv = apply_inverse(evoked_gen, inv_op, lambda2, method=method)
brain_inv = stc_inv.plot(**kwargs)



Projecting source estimate to sensor space...
[done]
4 projection items deactivated
Created an SSP operator (subspace dimension = 3)
4 projection items activated
SSP projectors applied...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
Applying inverse operator to ""...
    Picked 305 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  99.7% variance
[done]
Using control points [2.72983620e-10 3.45480986e-10 1.09652034e-09]
Reading labels from parcellation...
   read 34 labels from ../../../../data/tutorials/MNE-sample-data\subjects\sample\label\lh.aparc.annot
Reading labels from parcellation...
   read 34 labels from ../../../../data/tutorials/MNE-sample-data\subje

# Method as sLORETA SNR 10

In [20]:
seed = 42

# parameters for inverse method
method = 'sLORETA'
snr = 10.
lambda2 = 1.0 / snr ** 2

evoked_gen = simulate_evoked(fwd, stc_gen, evoked.info, cov, nave,
                             random_state=seed)

# Map the simulated sensor-space data to source-space using the inverse
# operator.
stc_inv = apply_inverse(evoked_gen, inv_op, lambda2, method=method)
brain_inv = stc_inv.plot(**kwargs)

Projecting source estimate to sensor space...
[done]
4 projection items deactivated
Created an SSP operator (subspace dimension = 3)
4 projection items activated
SSP projectors applied...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Applying inverse operator to ""...
    Picked 305 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained 100.0% variance
    sLORETA...
[done]
Using control points [0.12844106 0.1632574  0.54700837]


# Method as sLORETA SNR 1

In [21]:
seed = 42

# parameters for inverse method
method = 'sLORETA'
snr = 1.
lambda2 = 1.0 / snr ** 2

evoked_gen = simulate_evoked(fwd, stc_gen, evoked.info, cov, nave,
                             random_state=seed)

# Map the simulated sensor-space data to source-space using the inverse
# operator.
stc_inv = apply_inverse(evoked_gen, inv_op, lambda2, method=method)
brain_inv = stc_inv.plot(**kwargs)

Projecting source estimate to sensor space...
[done]
4 projection items deactivated
Created an SSP operator (subspace dimension = 3)
4 projection items activated
SSP projectors applied...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Applying inverse operator to ""...
    Picked 305 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  96.0% variance
    sLORETA...
[done]
Using control points [1.3776512 1.6437969 4.3271975]
